In [1]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from shapely.geometry import Polygon
from shapely.geometry import MultiPoint, MultiPolygon

In [2]:
import sys
# https://github.com/computationalpathologygroup/ASAP/releases
# Automated Slide Analysis Platform (ASAP) is an open source platform
if 'D:\\ACDC_LUNG_HISTOPATHOLOGY\\ASAP 1.9\\bin' not in sys.path:
    sys.path.append('D:\\ACDC_LUNG_HISTOPATHOLOGY\\ASAP 1.9\\bin')
import multiresolutionimageinterface as mir  # noqa pylint: disable=import-error
_READER = mir.MultiResolutionImageReader()
# slide=_READER.open('D:/ACDC_LUNG_HISTOPATHOLOGY/data/slides/1.tif')
# slide.close()
# import openslide


# %config InlineBackend.figure_format = 'retina'

In [ ]:
slide_path='D:/ACDC_LUNG_HISTOPATHOLOGY/data/slides/32.tif'
truth_path='D:/ACDC_LUNG_HISTOPATHOLOGY/data/masks/32_M.tif'

slide = openslide.open_slide(slide_path)
truth = openslide.open_slide(truth_path)

"Original Slide dimensions %dx%d" % slide.dimensions
"Original Truth dimensions %dx%d" % truth.dimensions

In [ ]:
thumbnail = slide.get_thumbnail((slide.dimensions[0] / 256, slide.dimensions[1] / 256))
thumbnail_truth=Image.fromarray(np.array(truth.get_thumbnail((truth.dimensions[0] / 256, truth.dimensions[1] / 256)) )[:,:,0]*255)

f, axes = plt.subplots(1, 2, figsize=(20, 10));
ax = axes.ravel()
ax[0].imshow(thumbnail);
ax[0].set_title('Slide %dx%d' % thumbnail.size)
ax[1].imshow(thumbnail_truth.convert('L'), cmap='gray');
ax[1].set_title('Truth %dx%d' % thumbnail_truth.size)
f.suptitle('Slide & Truth Thumbnails (downsampled 256x)');

In [ ]:
from skimage.filters import threshold_otsu

img = np.array(thumbnail.convert('L')) # convert to grayscale
thresh = threshold_otsu(img)
binary = img > thresh

In [ ]:

f, axes = plt.subplots(1, 3, figsize=(20, 10));
ax = axes.ravel();
ax[0].imshow(img, cmap='gray');
ax[0].set_title('Original');
ax[1].hist(img.ravel(), bins=256);
ax[1].set_title('Histogram of pixel values');
ax[1].axvline(thresh, color='r');
ax[2].imshow(binary, cmap='gray');
ax[2].set_title('Binary');

In [ ]:
from openslide.deepzoom import DeepZoomGenerator 

tiles = DeepZoomGenerator(slide, tile_size=256, overlap=0, limit_bounds=False)
tiles_truth = DeepZoomGenerator(truth, tile_size=256, overlap=0, limit_bounds=False)

In [ ]:
f, axes = plt.subplots(3, 2, figsize=(20, 10))
axes[0, 0].imshow(tiles.get_tile(tiles.level_count-1, (100, 280)));
axes[0, 0].set_title('Tumor Example');
axes[0, 1].imshow(Image.fromarray(np.array(tiles_truth.get_tile(tiles_truth.level_count-1, (100, 280)))[:,:,0]*255).convert('L'));
axes[0, 1].set_title('Truth mask (white=tumor, black=not_tumor)')

axes[1, 0].imshow(tiles.get_tile(tiles.level_count-1, (150, 300)));
axes[1, 0].set_title('Normal Example');
axes[1, 1].imshow(Image.fromarray(np.array(tiles_truth.get_tile(tiles_truth.level_count-1, (200, 300)))[:,:,0]*255).convert('L'));
axes[1, 1].set_title('Truth mask (white=tumor, black=not_tumor)')

axes[2, 0].imshow(tiles.get_tile(tiles.level_count-1, (240, 200)));
axes[2, 0].set_title('Tumor/Normal Boundary Example');
axes[2, 1].imshow(tiles_truth.get_tile(tiles_truth.level_count-1, (240, 200)));
axes[2, 1].set_title('Truth mask (white=tumor, black=not_tumor)');
plt.tight_layout();

In [ ]:
def get_gray_from_rgb(rgb_image):
    if type(rgb_image) is not np.ndarray:
        rgb_image=np.array(rgb_image)
    return cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY)

In [ ]:
def get_external_contours_from_grayscale_image(grayscale_image):
    image, contours, hierarchy = cv2.findContours(grayscale_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

In [ ]:
def get_bbox_from_contours(contours):
    return [cv2.boundingRect(c) for c in contours]

In [ ]:
slide_path='D:/ACDC_LUNG_HISTOPATHOLOGY/data/slides/70.tif'
slide = openslide.open_slide(slide_path)
thumbnail = slide.get_thumbnail((slide.dimensions[0] / 256, slide.dimensions[1] / 256))
thumbnail = cv2.cvtColor(np.array(thumbnail), cv2.COLOR_RGB2BGR) # читаем
gray = cv2.cvtColor(thumbnail, cv2.COLOR_BGR2GRAY) # rgb to gray
gray = cv2.GaussianBlur(gray, (3, 3), 0) # гаусоовый шум
edged = cv2.Canny(gray, 10, 250) # определяем края
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5)) 
closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
image, contours, hierarchy = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
for cont in contours:
    peri = cv2.arcLength(cont, True)
    approx = cv2.approxPolyDP(cont, 0.001 * peri, True)
    cv2.drawContours(thumbnail, [approx], -1, (0, 255, 0), 2)

In [ ]:
cv2.imshow("Output", thumbnail)
cv2.waitKey(0)

In [ ]:
len(contours)

In [ ]:
def normalize(image):    
    if type(image) is not np.ndarray:
        image=np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, *ab = cv2.split(image)
    clahe = cv2.createCLAHE(2., (8, 8))
    l = clahe.apply(l)
    image = cv2.merge([l, *ab])
    image = cv2.cvtColor(image, cv2.COLOR_LAB2BGR)
    return image

In [ ]:
# slide_path='D:/ACDC_LUNG_HISTOPATHOLOGY/data/slides/90.tif'
# slide = openslide.open_slide(slide_path)
# thumbnail = slide.get_thumbnail((slide.dimensions[0] / 256, slide.dimensions[1] / 256))


slide=_READER.open('D:/ACDC_LUNG_HISTOPATHOLOGY/data/slides/70.tif')
w,h=slide.getDimensions()
thumbnail=slide.getUCharPatch(0, 0, w//256, h//256, 8)
slide.close()
thumbnail = Image.fromarray(thumbnail).convert('RGB')
thumbnail=normalize(thumbnail)

gray = cv2.cvtColor(thumbnail, cv2.COLOR_BGR2GRAY) # rgb to gray
gray = cv2.GaussianBlur(gray, (3, 3), 0) # гаусоовый шум
edged = cv2.Canny(gray, 10, 250) # определяем края
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5)) 
closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
_, contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for cont in contours:
    peri = cv2.arcLength(cont, True)
    approx = cv2.approxPolyDP(cont, 0.001 * peri, True)
    cv2.drawContours(thumbnail, [approx], -1, (0, 255, 0), 2)

In [ ]:
cv2.imshow("Output", thumbnail)
cv2.waitKey(0)

In [ ]:
ccc=(contours[0]*256).reshape(-1,2)
poly=Polygon(ccc)

In [ ]:
poly

In [ ]:
Polygon([cv2.boundingRect(c)[:2] for c in contours[0]*256])

In [ ]:
cv2.pointPolygonTest()

In [ ]:
from shapely.geometry import MultiPoint

In [ ]:
m = MultiPoint([(0, 0), (1, 1), (1,2), (2,2)])

In [ ]:
for p in m:
    print(p)
    x, y = p.coords[0]
    print(x, y)

In [ ]:
pl = Polygon([(0, 0), (1, 1), (1,2), (2,1)])
pl

In [ ]:
for p in m.intersection(pl):
    print(p)

In [ ]:
# shape = (w,h)
mg = np.stack(np.meshgrid(np.arange(0, shape[0], 244), np.arange(0, shape[1], 244),
                          indexing='ij'), axis=-1).reshape(-1, 2)

In [ ]:
mp = MultiPoint(mg)

In [ ]:
ss=mp.intersection(poly.buffer(0))

In [ ]:
for i, s in enumerate(ss):
    x,y=s.coords[0]
    print(i)

In [ ]:
# jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj

In [3]:
slide_path='D:/ACDC_LUNG_HISTOPATHOLOGY/data/slides/32.tif'

In [4]:
def get_shape(slide_path):
    reader=mir.MultiResolutionImageReader()
    slide=reader.open(slide_path)
    shape=slide.getDimensions()
    slide.close()
    return shape

def get_thumbnail(slide_path, zoom=8):
    reader=mir.MultiResolutionImageReader()
    slide=reader.open(slide_path)
    shape=slide.getDimensions()
    thumbnail=slide.getUCharPatch(0, 0, shape[0]//2**zoom, shape[1]//2**zoom, zoom)
    slide.close()
    return thumbnail

def normalize(slide):    
    if type(slide) is not np.ndarray:
        slide=np.array(slide)
    slide = cv2.cvtColor(slide, cv2.COLOR_BGR2LAB)
    l, *ab = cv2.split(slide)
    clahe = cv2.createCLAHE(2., (8, 8))
    l = clahe.apply(l)
    slide = cv2.merge([l, *ab])
    slide = cv2.cvtColor(slide, cv2.COLOR_LAB2BGR)
    return slide

def get_aprox_contours(slide):
    gray = cv2.cvtColor(slide, cv2.COLOR_BGR2GRAY) # rgb to gray
    gray = cv2.GaussianBlur(gray, (3, 3), 0) # гаусоовый шум
    edged = cv2.Canny(gray, 10, 250) # определяем края
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5)) 
    closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
    _, contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cont in contours:
        yield cv2.approxPolyDP(cont, 0.001 * cv2.arcLength(cont, True), True).reshape(-1,2)
        
def get_grid_points(shape, patch_size):
    mg = np.stack(np.meshgrid(np.arange(0, shape[0], patch_size[0]), np.arange(0, shape[1], patch_size[1]), 
                              indexing='ij'), axis=-1).reshape(-1, 2)
    return MultiPoint(mg)

def to_multi2(contours, zoom):
    return MultiPolygon([p*2**zoom, []] for p in contours).buffer(0)

def get_test_points(grid_points, multi_poly):
    for point in grid_points.intersection(multi_poly):
        yield point.coords[0]

In [7]:
shape=get_shape(slide_path)
thumbnail=get_thumbnail(slide_path, zoom=4)
thumbnail=normalize(thumbnail)
contours=list(get_aprox_contours(thumbnail))
multi_poly=to_multi2(contours, zoom=4)
grid_points=get_grid_points(shape, (224,224))
test_points=list(get_test_points(grid_points, multi_poly))
# for point in grid_points.intersection(multi_poly):
#     tra ta ta

In [8]:
s=len(test_points)
s

In [9]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision.transforms import functional as F
import torch
from torchvision.models import inception_v3
from ignite.engine import create_supervised_evaluator, create_supervised_trainer
from torch.optim import Adam
from torch import nn

In [10]:
_READER=mir.MultiResolutionImageReader()
class TestDataset(Dataset):
    def __init__(self, slide_path, _list, patch_size, zoom):
        self._list=_list
        self.slide_path=slide_path
        self.patch_size=patch_size
        self.zoom=zoom
    def __len__(self):
        return len(self._list)
    def __getitem__(self, index):
        slide = _READER.open(self.slide_path)
        slide_patch=slide.getUCharPatch(startX=int(self._list[index][0]), startY=int(self._list[index][1]),             
                                        height=self.patch_size[0], width=self.patch_size[1], level=self.zoom)
        slide_patch = Image.fromarray(slide_patch).convert('RGB')
        slide_patch = F.to_tensor(slide_patch)
        
        return slide_patch
        
        
        

In [11]:
test_d=TestDataset(slide_path, test_points, (224,224), 0)
test_loader=DataLoader(test_d, batch_size=12, num_workers=6, shuffle=False)

In [17]:
len(test_loader)

5032

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [13]:
model = inception_v3(num_classes=1, aux_logits=False)
model = model.to(device)

model_state_dict = torch.load("D:/ACDC_LUNG_try2/checkpoints/19-02-22/model_inception_2277_acc=0.967.pth")
model.load_state_dict(model_state_dict)
model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [14]:
optimizer = Adam(model.parameters(), lr=5e-5)
criterion = nn.BCEWithLogitsLoss()
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)

In [15]:
evaluator = create_supervised_evaluator(model, device=device)

In [20]:
len(test_loader)

5032

In [22]:
len(test_loader)*12

60384

In [23]:
import pickle

In [1]:
from ignite.metrics import Precision, Recall, Accuracy

In [361]:
import torch
def confmatrix(output):
    nb_classes=2
    y_pred, y_true, *_ =output
    
    print(y_pred)
    y_pred=y_pred.round()
    print(y_pred)
    y_pred = torch.zeros(y_pred.shape[0], nb_classes, device=y_pred.device) #.scatter_(1, y_pred.long(), 1)
    print(y_pred)
    y_true = torch.zeros(nb_classes, y_true.shape[1], device=y_true.device).scatter_(0, y_true, 1)
    print(y_true)
    state = (y_true @ y_pred).double()
    return state / state.sum([0, 1])

In [415]:
import torch
a=torch.tensor([[0.6], [0.6], [0.6], [0.3], [0.1], [0.7]])
b=torch.tensor([[1], [1], [1], [1], [0], [1]])

In [389]:
def conf_matrix(output, nb_classes=2):
    y_pred, y_true, *_=output
    y_pred=y_pred.round().long()
    y_pred=torch.zeros(y_pred.shape[0], nb_classes).scatter_(1, y_pred, 1)
    y_true=torch.t(torch.zeros(y_true.shape[0], nb_classes).scatter_(1, y_true, 1))
    cm=(y_true@y_pred).double()
    return cm/cm.sum([0,1])

In [416]:
output=a,b
cm=conf_matrix(output)

In [392]:
cm.numpy()

array([[0.33333333, 0.        ],
       [0.33333333, 0.33333333]])

In [417]:
class ConfusionMatrix:
    def __init__(self, output, nb_classes=2):
        self.output=output
        self.nb_classes=nb_classes
    
    def __call__(self):
        y_pred, y_true, *_ = self.output
        y_pred = y_pred.round().long()
        y_pred = torch.zeros(y_pred.shape[0], self.nb_classes).scatter_(1, y_pred, 1)
        y_true = torch.t(torch.zeros(y_true.shape[0], self.nb_classes).scatter_(1, y_true, 1))
        cm = (y_true @ y_pred).double()
        
        return cm / cm.sum([0, 1])

In [418]:
cm=ConfusionMatrix(output)

In [419]:
cm.__call__()

tensor([[0.1667, 0.0000],
        [0.1667, 0.6667]], dtype=torch.float64)